# MLS3 Hardware-Aware Design - Colab 运行环境

这个 notebook 提供了在 Google Colab 上运行 MLS3 多文件项目的完整设置流程。

## 🚀 快速开始

按顺序运行以下单元格即可完成整个流程：
1. 从 GitHub 克隆项目
2. 环境设置和依赖安装
3. 训练基线模型（可选，如果已有模型）
4. 运行优化流程
5. 查看结果和下载输出

## 📋 项目仓库

- **GitHub**: https://github.com/ishikisiko/MSL.git
- **项目路径**: MSL/MLS3

## 📥 步骤 1: 从 GitHub 克隆项目

In [ ]:
# 克隆项目仓库
print("正在从 GitHub 克隆项目...")
!git clone https://github.com/ishikisiko/MSL.git
print("✓ 克隆完成")

# 切换到 MLS3 目录
%cd MSL/MLS3

# 显示当前目录内容
print("\n当前目录内容：")
!ls -la

## 🔧 步骤 2: 环境设置和依赖安装

In [ ]:
# 检查当前目录和文件
!pwd
!ls -la

In [ ]:
# 安装依赖（使用项目的 requirements.txt）
print("安装 Python 依赖包...")

print("步骤 1/2: 升级 pip 以获取兼容 wheel...")
!python -m pip install --upgrade pip

print("步骤 2/2: 安装项目锁定的依赖...")
!python -m pip install --quiet -r requirements.txt
# 可选：仅当计划使用 %lprun profiling magic 时安装 line_profiler
# !python -m pip install --quiet line_profiler

# 验证安装
import numpy as np
import tensorflow as tf
import tensorflow_model_optimization as tfmot
print("\n✓ 依赖安装完成")
print(f"✓ NumPy: {np.__version__} (需求: 1.26.4)")
print(f"✓ TensorFlow: {tf.__version__} (需求: 2.15.1)")
print(f"✓ TF-MOT: {tfmot.__version__}")

In [ ]:
# 检查 GPU 可用性
import tensorflow as tf

print("TensorFlow 版本:", tf.__version__)
print("GPU 可用:", tf.config.list_physical_devices('GPU'))

if tf.config.list_physical_devices('GPU'):
    print("\n✓ GPU 已启用，训练将更快！")
    # 设置内存增长
    for gpu in tf.config.list_physical_devices('GPU'):
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("\n⚠ 未检测到 GPU，将使用 CPU 运行")
    print("建议：在 Colab 菜单中选择 '运行时' -> '更改运行时类型' -> 选择 GPU")

In [ ]:
# 快速测试模块导入
print("测试模块导入...\n")

try:
    import part1_baseline_model
    print("✓ part1_baseline_model 导入成功")
except ImportError as e:
    print(f"✗ part1_baseline_model 导入失败: {e}")

try:
    import part2_optimizations
    print("✓ part2_optimizations 导入成功")
except ImportError as e:
    print(f"✗ part2_optimizations 导入失败: {e}")

try:
    import performance_profiler
    print("✓ performance_profiler 导入成功")
except ImportError as e:
    print(f"✗ performance_profiler 导入失败: {e}")

try:
    import part3_modeling
    print("✓ part3_modeling 导入成功")
except ImportError as e:
    print(f"✗ part3_modeling 导入失败: {e}")

try:
    import part3_deployment
    print("✓ part3_deployment 导入成功")
except ImportError as e:
    print(f"✗ part3_deployment 导入失败: {e}")

print("\n✓ 所有模块导入完成")

## 🎯 步骤 3: 训练基线模型（可选）

如果你已经有训练好的 `baseline_mobilenetv2.keras` 文件，可以跳过这一步。

In [ ]:
# 检查是否已有基线模型
import os

if os.path.exists('baseline_mobilenetv2.keras'):
    print("✓ 发现已训练的基线模型，跳过训练步骤")
    SKIP_TRAINING = True
else:
    print("⚠ 未发现基线模型，需要训练")
    SKIP_TRAINING = False

In [ ]:
# 训练基线模型（如果需要）
# 警告：这将花费 30-60 分钟！

if not SKIP_TRAINING:
    print("开始训练基线模型...")
    print("预计时间：30-60 分钟（使用 GPU）")
    print("\n你可以选择：")
    print("1. 等待完整训练完成（推荐）")
    print("2. 或者在本地训练后上传 baseline_mobilenetv2.keras 文件")
    
    # 运行训练
    !python part1_baseline_model.py
    
    print("\n✓ 基线模型训练完成")
else:
    print("跳过训练步骤")

## ⚙️ 步骤 4: 运行完整优化流程

这将执行以下操作：
1. 创建三种优化模型（延迟/内存/能耗）
2. 应用量化技术
3. 性能基准测试
4. 生成对比报告
5. Track B 仿真演示
6. Track A 部署转换演示

In [ ]:
# 运行完整优化流程
# 预计时间：20-40 分钟

print("开始运行优化流程...")
print("预计时间：20-40 分钟\n")

!python run_optimizations.py

print("\n✓ 优化流程完成")

## 📊 步骤 5: 查看结果

In [ ]:
# 列出生成的文件
print("生成的优化模型：")
!ls -lh optimized_models/

print("\n结果文件：")
!ls -lh results/

print("\n日志文件：")
!ls -lh logs/

In [ ]:
# 快速性能对比可视化
import matplotlib.pyplot as plt
import numpy as np

# 示例数据（实际数据将从运行结果中提取）
models = ['Baseline', 'Latency\nOptimized', 'Memory\nOptimized', 'Energy\nOptimized']

# 创建子图
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 模型大小对比
ax = axes[0, 0]
sizes = [100, 50, 25, 60]  # 示例数据
ax.bar(models, sizes, color=['#3498db', '#2ecc71', '#e74c3c', '#f39c12'])
ax.set_ylabel('Model Size (MB)')
ax.set_title('Model Size Comparison')
ax.grid(axis='y', alpha=0.3)

# 延迟对比
ax = axes[0, 1]
latencies = [100, 45, 70, 55]  # 示例数据
ax.bar(models, latencies, color=['#3498db', '#2ecc71', '#e74c3c', '#f39c12'])
ax.set_ylabel('Latency (ms)')
ax.set_title('Inference Latency Comparison')
ax.grid(axis='y', alpha=0.3)

# 能耗对比
ax = axes[1, 0]
energy = [100, 60, 75, 40]  # 示例数据
ax.bar(models, energy, color=['#3498db', '#2ecc71', '#e74c3c', '#f39c12'])
ax.set_ylabel('Energy (mJ)')
ax.set_title('Energy Consumption Comparison')
ax.grid(axis='y', alpha=0.3)

# 准确率对比
ax = axes[1, 1]
accuracy = [85.5, 83.2, 81.0, 84.1]  # 示例数据
ax.bar(models, accuracy, color=['#3498db', '#2ecc71', '#e74c3c', '#f39c12'])
ax.set_ylabel('Accuracy (%)')
ax.set_title('Classification Accuracy Comparison')
ax.set_ylim([75, 90])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('results/performance_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ 性能对比图已保存到 results/performance_comparison.png")

## 💾 步骤 6: 下载结果文件

In [ ]:
# 打包所有结果文件
import shutil
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
archive_name = f"MLS3_results_{timestamp}"

print("打包结果文件...")

# 创建临时目录
!mkdir -p /content/export/{archive_name}

# 复制重要文件
!cp -r optimized_models /content/export/{archive_name}/
!cp -r results /content/export/{archive_name}/
!cp -r logs /content/export/{archive_name}/
!cp baseline_mobilenetv2.keras /content/export/{archive_name}/ 2>/dev/null || true

# 创建 ZIP 文件
%cd /content/export
!zip -r {archive_name}.zip {archive_name}
%cd /content/MLS3

print(f"\n✓ 结果已打包到: /content/export/{archive_name}.zip")
print(f"文件大小: ", end="")
!du -h /content/export/{archive_name}.zip

In [ ]:
# 下载结果压缩包
from google.colab import files

zip_path = f"/content/export/{archive_name}.zip"
print(f"下载 {archive_name}.zip...")
files.download(zip_path)
print("\n✓ 下载完成！")

## 🔍 单独运行特定部分（可选）

如果你只想运行某个特定部分，可以使用以下单元格。

In [ ]:
# 选项 A: 仅创建优化模型
from part2_optimizations import (
    create_latency_optimized_model,
    create_memory_optimized_model,
    create_energy_optimized_model
)

# 创建延迟优化模型
latency_model = create_latency_optimized_model(
    input_shape=(128, 128, 3),
    num_classes=10,
    alpha=0.5
)
print("✓ 延迟优化模型已创建")

# 查看模型架构
latency_model.summary()

In [ ]:
# 选项 B: 仅量化模型
from part2_optimizations import dynamic_range_quantization
from tensorflow import keras

# 加载模型
model = keras.models.load_model('baseline_mobilenetv2.keras')

# 应用动态量化
dynamic_range_quantization(model, save_path='optimized_models/test_quantized.tflite')
print("✓ 量化完成")

In [ ]:
# 选项 C: 仅性能分析
from performance_profiler import profile_model_comprehensive, print_profiling_results
from tensorflow import keras
from part1_baseline_model import load_and_preprocess_data

# 加载模型和数据
model = keras.models.load_model('baseline_mobilenetv2.keras')
_, _, test_ds = load_and_preprocess_data(batch_size=64)

# 性能分析
platform_config = {
    "power_budget_w": 5.0,
    "memory_budget_mb": 1024,
    "tdp_watts": 10.0,
}

results = profile_model_comprehensive(model, test_ds, platform_config)
print_profiling_results(results, "Baseline Model")

## 📝 生成分析报告

根据运行结果生成 Part 4 分析报告的数据。

In [ ]:
# 汇总关键指标
import json

summary = {
    "project": "MLS3 Hardware-Aware Design",
    "date": datetime.now().strftime("%Y-%m-%d"),
    "models": [
        {"name": "baseline", "path": "baseline_mobilenetv2.keras"},
        {"name": "latency_optimized", "path": "optimized_models/latency_optimized.keras"},
        {"name": "memory_optimized", "path": "optimized_models/memory_optimized.keras"},
        {"name": "energy_optimized", "path": "optimized_models/energy_optimized.keras"},
    ],
    "quantized_models": [
        {"name": "dynamic_quantization", "path": "optimized_models/latency_optimized_dynamic.tflite"},
        {"name": "ptq_int8", "path": "optimized_models/latency_optimized_ptq_int8.tflite"},
    ],
    "deployment": [
        {"name": "tflite", "path": "optimized_models/baseline.tflite"},
        {"name": "tflite_micro", "path": "optimized_models/baseline_data.cc"},
    ]
}

# 保存摘要
with open('results/project_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("✓ 项目摘要已保存到 results/project_summary.json")
print("\n摘要内容：")
print(json.dumps(summary, indent=2))

## 🎉 完成！

### 生成的文件：

**优化模型：**
- `optimized_models/latency_optimized.keras` - 延迟优化模型
- `optimized_models/memory_optimized.keras` - 内存优化模型
- `optimized_models/energy_optimized.keras` - 能耗优化模型

**量化模型：**
- `optimized_models/*_dynamic.tflite` - 动态量化模型
- `optimized_models/*_ptq_int8.tflite` - INT8 PTQ 模型

**部署文件：**
- `optimized_models/*.tflite` - TFLite 格式
- `optimized_models/*_data.cc` - TFLite Micro C 数组

**结果和日志：**
- `results/` - 性能指标和对比报告
- `logs/` - 训练和执行日志

### 下一步：
1. 下载所有结果文件（已打包到 ZIP）
2. 分析性能对比数据
3. 编写 Part 4 综合分析报告
4. 准备演示材料

### 提示：
- 使用 `files.download()` 下载单个文件
- 所有结果已打包到 ZIP 文件中
- 可以在 Colab 文件浏览器中查看所有生成的文件
- 如需重新运行，请 "运行时" -> "重启并运行所有单元格"